# DiabaticRetinopathydiseasedetectiont

Instruction:
1. install dependencies like 
    tensorflow, keras for CNN modeling
    PILLOW and cv2 for image processing
    Numpy mathplot , skkitplot sklearn io for data and image transformation and  result graph plots etc
2. connect google drive before runing the code
3. run the code.
4.you can test enhanced image by code mention lin last code cell

### install dependencies like tensorflow

In [ ]:
!pip install tensorflow

### Connect Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive

### Complete code to run 

In [ ]:
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from tensorflow import lite
import tensorflow as tf
# load and evaluate a saved model
import numpy as np
from keras.models import load_model
from keras.preprocessing import image
from matplotlib import pyplot as plt
import cv2
import os
from google.colab.patches import cv2_imshow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn import preprocessing
from keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout
from sklearn.model_selection import train_test_split
import math
from keras.preprocessing import image
from PIL import Image
from matplotlib import cm
from PIL import ImageEnhance
import io
import numpy as np
from keras.layers import Input
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.models import Model
from keras.layers import GlobalAveragePooling2D,MaxPooling2D
from keras.layers import Dense,Flatten,SpatialDropout2D
from keras.layers.merge import concatenate

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import History
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import pandas as pd
%matplotlib inline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import r2_score, roc_auc_score, roc_curve
from scipy import stats  # For in-built method to get PCC
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve

class Diabatic_Retino_Detection():
    def __init__(self, dataset_path="/content/drive/MyDrive/retinopathy-dataset",enhanceddataset_path="/content/drive/MyDrive/newretinopathy-dataset",data_path="/content/drive/MyDrive/",models_path="'/content/drive/MyDrive/model/'"):
        self.dataset_path  = dataset_path
        self.enhanceddataset_path=enhanceddataset_path
        self.models_path=models_path
        self.data_path=data_path
        self.predictionsclasses = ['nosymptoms', 'symptoms']

#---------------Image acquisition per image---------- details: it read image and convert it into digital , do shaping and resizing etc.
    def image_acquisition(self,image_file, gray_scale=False):
        image_src= cv2.imread( image_file)
        if gray_scale:
          image_rgb = cv2.cvtColor(image_src, cv2.COLOR_BGR2GRAY)
        else:
          image_rgb = cv2.cvtColor(image_src, cv2.COLOR_BGR2RGB)
        TARGET_PIXEL_AREA = 100000.0
        img=image_rgb
        ratio = float(img.shape[1]) / float(img.shape[0])
        new_h = int(math.sqrt(TARGET_PIXEL_AREA / ratio) + 0.5)
        new_w = int((new_h * ratio) + 0.5)
        img2 = cv2.resize(img, (new_w,new_h))
        return img2

#---------------Image enhance contrast per image---------- details: it it enhance the contrast in per image it is a sub function of function mentioned below this function etc.
    def enhance_contrast(self,image_matrix, bins=256):
        image_flattened = image_matrix.flatten()
        image_hist = np.zeros(bins)

        # frequency count of each pixel
        for pix in image_matrix:
            image_hist[pix] += 1

        # cummulative sum
        cum_sum = np.cumsum(image_hist)
        norm = (cum_sum - cum_sum.min()) * 255
        # normalization of the pixel values
        n_ = cum_sum.max() - cum_sum.min()
        uniform_norm = norm / n_
        uniform_norm = uniform_norm.astype('int')

        # flat histogram
        image_eq = uniform_norm[image_flattened]
        # reshaping the flattened matrix to its original shape
        image_eq = np.reshape(a=image_eq, newshape=image_matrix.shape)

        return image_eq

#---------------Image equalize per image---------- details: it it enhance and equalize per image and preprocess it for next step of reshaping for dataset etc.
    def equalize_this(self,image_file,saveimage_file, with_plot=False, gray_scale=False, bins=256):
        image_src = self.image_acquisition(image_file, gray_scale=False)
        if not gray_scale:
            r_image = image_src[:, :, 0]
            g_image = image_src[:, :, 1]
            b_image = image_src[:, :, 2]

            r_image_eq = self.enhance_contrast(image_matrix=r_image)
            g_image_eq = self.enhance_contrast(image_matrix=g_image)
            b_image_eq = self.enhance_contrast(image_matrix=b_image)

            image_eq = np.dstack(tup=(r_image_eq, g_image_eq, b_image_eq))
            cmap_val = None
        else:
            image_eq = self.enhance_contrast(image_matrix=image_src)
            cmap_val = 'gray'

        #if with_plot:
        fig = plt.figure(figsize=(10, 20))

        #ax1 = fig.add_subplot(2, 2, 1)
        #ax1.axis("off")
        #ax1.title.set_text('Original')
        #ax2 = fig.add_subplot(2, 2, 2)
        #ax2.axis("off")
        #ax2.title.set_text("Equalized")

        #ax1.imshow(image_src, cmap=cmap_val)
        #ax2.imshow(image_eq, cmap=cmap_val)
        plt.imsave(saveimage_file, np.uint8(image_eq), cmap=cmap_val)
                    # load the image
        image = Image.open(saveimage_file)
            # convert image to numpy array
        data = np.asarray(image)
        #print(type(data))
            # summarize shape
        #print(data.shape)

            # create Pillow image
        image2 = Image.fromarray(data)
        #print(type(image2))

        # summarize image details
        #print(image2.mode)
        #print(image2.size)
        #print(type(image2))
        #img = Image.open(fh, mode='r')
        roi_img = image2

        img_byte_arr = io.BytesIO()
        roi_img.save(img_byte_arr, format='jpeg')
        img_byte_arr = img_byte_arr.getvalue()
        #print(type(img_byte_arr))
        return img_byte_arr

#---------------Image reshaping per image---------- details: it reshape the image array to store in dataset.
    def image_reshaping(self,image):
        #new_im = Image.fromarray(image)
        new_im = np.array(image)
        return new_im

#---------------Imagedata preprocessing ---------- details: this function convert all the input dataset images into enhance images and save image arrays of byte and extract labels and save them into .
    def imagedata_preprocessing(self):
        print("Image Data preprocessing started...")
        print("Image Data acquisition started...")
        # preprocessing
        imgs_path = self.dataset_path
        data = []
        labels = []
        labelclasses = os.listdir(imgs_path)
        for labelclass in labelclasses:
            img_path = os.path.join(imgs_path, str(labelclass))  # 0-42
            saveimg_path = os.path.join(self.enhanceddataset_path, str(labelclass))  # 0-42
            imgs =os.listdir(img_path)
            for img in imgs:
                image_file=img_path+'/'+img
                saveimagefile=saveimg_path+'/'+img
                #image_file=os.path.join(imgs_path, img)
                image_eq=self.equalize_this(image_file,saveimagefile, with_plot=False, gray_scale=False, bins=256)
                im = self.image_reshaping(image_eq)
                data.append(im)
                labels.append(labelclass)
                print('label',labelclass,'filename',img)
        print("Image Data successfully preprocessed")
        self.data = data
        self.labels = labels

#---------------Imagedata preprocessing ---------- details:this function convert imagedataset array return by preprocessing funtion into numpy array and change the categorical labels into numeric this process is called transformation of data for modeling purpose.
    def imagedata_transformation(self):
        #print(self.data)
        #x_train= np.asarray(self.data)
        #y_train = np.asarray(self.labels)
        #print("training shape: ", x_train.shape, y_train.shape)
        #le = preprocessing.LabelEncoder()
        #y_train=le.fit_transform(y_train)
        #self.x_train= x_train
        #self.y_train=y_train
        train_data_path = self.enhanceddataset_path
        train_datagen = ImageDataGenerator(rescale=1./255,
                                      rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      validation_split=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest')
        training_data = train_datagen.flow_from_directory(train_data_path, 
                                      target_size=(150,150),
                                      batch_size=32,
                                      class_mode='binary',
                                      subset='training')
        valid_data = train_datagen.flow_from_directory(train_data_path, 
                                      target_size=(150,150),
                                      batch_size=32,
                                      class_mode='binary',
                                      subset='validation')

        for i in os.listdir(train_data_path):
          print(str(len(os.listdir(train_data_path+'/'+i))) +" "+ i +" images")
        
        self.training_data=training_data
        self.valid_data=valid_data
        self.valid_datax=valid_data[0][0]
        self.valid_datalabels= valid_data.labels
        self.valid_dataclasses= valid_data.class_indices.keys()
        #print(valid_data[0][0])
        print('Image data transformation done successfully.')

#---------------CNN Model configration---------- details: this function is about model configuration in which we set model checkpoints set number of epochs number of layers and type of layer number of outputs and number of iterations in model and model path .
    def model_configration(self):
        image_size = (64, 64)
        print("Convolutional Model configration started...")
        model_path = self.models_path+'test.h5'
        print(model_path)
        model = keras.models.Sequential([
                                    keras.layers.Conv2D(filters=32, kernel_size=3, input_shape=[150, 150, 3]),
                                    keras.layers.MaxPooling2D(pool_size=(2,2)),
                                    keras.layers.Conv2D(filters=64, kernel_size=3),
                                    keras.layers.MaxPooling2D(pool_size=(2,2)),
                                    keras.layers.Conv2D(filters=128, kernel_size=3),
                                    keras.layers.MaxPooling2D(pool_size=(2,2)),                                    
                                    keras.layers.Conv2D(filters=256, kernel_size=3),
                                    keras.layers.MaxPooling2D(pool_size=(2,2)),
 
                                    keras.layers.Dropout(0.5),                                                                        
                                    keras.layers.Flatten(), 
                                    keras.layers.Dense(units=128, activation='relu'), 
                                    keras.layers.Dropout(0.1),                                    
                                    keras.layers.Dense(units=256, activation='relu'),                                    
                                    keras.layers.Dropout(0.25),                                    
                                    keras.layers.Dense(units=2, activation='softmax') ])
        model.compile(optimizer = Adam(lr=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.summary()
        print("Convolutional Model successfully configured.")
        self.model=model
#---------------CNN model Performance graph---------- details: this function is about  for performance and evaluation graphs .   
    def plot_model_history(self,history):
        #history=self.history
        print(history.history.keys())
        # summarize history for accuracy
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()

        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()
#---------------model testing performance graphs---------- details: this function test image after enhancing and show the labels .   
    def results(self,target_test, predicted_test,ModelName,labels):
       target_names = labels
       print(classification_report(target_test, predicted_test, target_names=target_names))
       y_test = target_test
       preds = predicted_test
       rms = np.sqrt(np.mean(np.power((np.array(y_test) - np.array(preds)), 2)))
       score = r2_score(y_test, preds)
       mae = mean_absolute_error(y_test, preds)
       mse = mean_squared_error(y_test, preds)
       pearson_coef, p_value = stats.pearsonr(y_test, preds)

       print("root mean square:", rms)
       print("score:", score)
       print("mean absolute error:", mae)
       print("mean squared error:", mse)
       print("pearson_coef:", pearson_coef)
       print("p_value:", p_value)
       print("=======================================================================\n\n")
       skplt.metrics.plot_confusion_matrix(
        y_test,
        preds,
        figsize=(10, 6), title="Confusion matrix\n Deposite Category "+ModelName)
       plt.xlim(-0.5, len(np.unique(y_test)) - 0.5)
       plt.ylim(len(np.unique(y_test)) - 0.5, -0.5)
       plt.savefig('cvroc.png')
       plt.show()

#---------------CNN model training---------- details: this function is about training model by giving input training dataset and labels which we got from above funcitons and set validation ratio 0.2 for performance and evaluation .
    def train_model(self):
        print("Convolutional Model training started...")
        training_data=self.training_data
        valid_data=self.valid_data
        model= self.model
        history = History()
        callbacks = [EarlyStopping(monitor='val_loss', patience=5),ModelCheckpoint(filepath=self.models_path+'best_model.h5', monitor='val_loss', save_best_only=True),history]
        history = model.fit_generator(training_data,
                                      steps_per_epoch = 8000/32,
                                      epochs = 1000,
                                      validation_data = valid_data,
                                      validation_steps = 64,
                                      use_multiprocessing = True,
                                      workers = 8,
                                      callbacks=callbacks)
        
        self.plot_model_history(history)
        model.save(self.models_path+'model.hd5')
        print("Convolutional Model successfully trained.")
        print("Convolutional Model Performance & testing results")
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()

        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()
        self.history=history
        test_datax= self.valid_datax
        predicted_test=model.fit(test_datax)
        target_test=self.valid_datalabels
        ModelName= "Convolutional Neural Network"
        labels=self.valid_dataclasses
        self.results(target_test, predicted_test,ModelName,labels)

    def train_previousmodel(self):
        print("Convolutional Model training started...")
        model_path='/content/drive/MyDrive/model/model.hd5'
        model = load_model(model_path)
        training_data=self.training_data
        valid_data=self.valid_data
        #model= self.model
        history = History()
        callbacks = [EarlyStopping(monitor='val_loss', patience=5),ModelCheckpoint(filepath=self.models_path+'best_model.h5', monitor='val_loss', save_best_only=True),history]
        history = model.fit_generator(training_data,
                                      steps_per_epoch = 8000/32,
                                      epochs = 1000,
                                      validation_data = valid_data,
                                      validation_steps = 64,
                                      use_multiprocessing = True,
                                      workers = 8,
                                      callbacks=callbacks)
        
        self.plot_model_history(history)
        model.save(model_path)
        history
        print("Convolutional Model successfully trained.")
        print("Convolutional Model Performance & testing results")
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()

        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()
        self.history=history
        test_datax= self.valid_datax
        predicted_test=model.fit(test_datax)
        target_test=self.valid_datalabels
        ModelName= "Convolutional Neural Network"
        labels=self.valid_dataclasses
        self.results(target_test, predicted_test,ModelName,labels)
#---------------hd5 model to tf lite model conversion---------- details: this function converts the model into tf lite version for lite applications .     
    def convert_hd5model_to_tflite(self):
        print("Convolutional Model conversion from to tflite started...")
        models = tf.keras.models.load_model('/content/drive/MyDrive/model/model.hd5')
        converter = lite.TFLiteConverter.from_keras_model(models)
        print(converter)
        model = converter.convert()
        tfmodel_path=self.models_path+'model.tflite'
        assert isinstance(model, object)
        file = open(tfmodel_path, 'wb').write(model)
        print("Convolutional Model successfully converted from to tflite...")

#---------------model testing---------- details: this function test image after enhancing and show the labels .          
    def test_model(self,Image_test_path):
        print("Image testing started...")
        # load model\
        model_path=self.models_path+'model.hd5'
        model = load_model(model_path)
        # summarize model.
        # model.summary()
        # load dataset
        image1 = image.load_img(Image_test_path, target_size=(150, 150))
        image1.save('test.jpeg')
        # image1
        #input_arr = image.img_to_array(image1)
        image1=self.equalize_this('test.jpeg','test.jpeg' , with_plot=False, gray_scale=False, bins=256)
        input_arr = self.image_reshaping(image1) # Convert single image to a batch
        #input_arr = image.img_to_array(input_arr)
        #input_arr
        predictions1 = model.predict(input_arr)
        print(predictions1)
        probabilities1 = model.predict_proba(input_arr)
        print(probabilities1)
        predictionsclasses = self.predictionsclasses
        probabilitiestest = list(probabilities1)
        testprob = max(probabilitiestest)
        print(testprob.tolist())
        max_value = max(testprob)
        print(max_value)
        for i in range(len(testprob)):
            if testprob[i] == max_value:
                max_index = i
                print(predictionsclasses[max_index])
        print("Image testing successfully completed.")

if __name__ == "__main__":
    Diabatic_Retino_Detection = Diabatic_Retino_Detection()
    #Diabatic_Retino_Detection.imagedata_preprocessing()
    Diabatic_Retino_Detection.imagedata_transformation()
    Diabatic_Retino_Detection.model_configration()
    Diabatic_Retino_Detection.train_model()
    #Diabatic_Retino_Detection.train_previousmodel()
    Image_test1 = '/content/drive/MyDrive/retinopathy-dataset/nosymptoms/10265_left.jpeg'
    Image_test2 = '/content/drive/MyDrive/retinopathy-dataset/nosymptoms/10643_left.jpeg'
    Image_test3 = '/content/drive/MyDrive/retinopathy-dataset/symptoms/10030_left.jpeg'
    Image_test4 = '/content/drive/MyDrive/retinopathy-dataset/symptoms/10030_right.jpeg'
    Diabatic_Retino_Detection.test_model(Image_test1)
    Diabatic_Retino_Detection.test_model(Image_test2)
    Diabatic_Retino_Detection.test_model(Image_test3)
    Diabatic_Retino_Detection.test_model(Image_test4)

### code to test the image enhancement process

In [ ]:
from matplotlib import pyplot as plt
import cv2
import numpy as np
from keras.preprocessing import image
from PIL import Image
import numpy as np
from PIL import Image
from matplotlib import cm
import math
# load a fundus image, normalized into 0-1 range (not 0-255)
# such as one from the IDRiD dataset  (assuming you already have it on disk)
#dset = IDRiD('./data/IDRiD_segmentation')
#img_id, img, labels = dset.sample()
#print("using image", img_id)
#import cv2

def read_this(image_file, gray_scale=False):
    image_src = cv2.imread(image_file)
    print(type(image_src))
    if gray_scale:
        image_rgb = cv2.cvtColor(image_src, cv2.COLOR_BGR2GRAY)
    else:
        image_rgb = cv2.cvtColor(image_src, cv2.COLOR_BGR2RGB)
    TARGET_PIXEL_AREA = 100000.0
    img=image_rgb
    ratio = float(img.shape[1]) / float(img.shape[0])
    new_h = int(math.sqrt(TARGET_PIXEL_AREA / ratio) + 0.5)
    new_w = int((new_h * ratio) + 0.5)
    img2 = cv2.resize(img, (new_w,new_h))
    print(type(img2))
    return img2
def enhance_contrast(image_matrix, bins=256):
    image_flattened = image_matrix.flatten()
    image_hist = np.zeros(bins)

    # frequency count of each pixel
    for pix in image_matrix:
        image_hist[pix] += 1

    # cummulative sum
    cum_sum = np.cumsum(image_hist)
    norm = (cum_sum - cum_sum.min()) * 255
    # normalization of the pixel values
    n_ = cum_sum.max() - cum_sum.min()
    uniform_norm = norm / n_
    uniform_norm = uniform_norm.astype('int')

    # flat histogram
    image_eq = uniform_norm[image_flattened]
    # reshaping the flattened matrix to its original shape
    image_eq = np.reshape(a=image_eq, newshape=image_matrix.shape)
    print(type(image_eq))
    return image_eq

import matplotlib.pyplot as plt
def demorgb(image,colormap):
  # Get the color map by name:
  
  cm = plt.get_cmap(colormap)
  # Apply the colormap like a function to any array:
  colored_image = cm(image)
  # Obtain a 4-channel image (R,G,B,A) in float [0, 1]
  # But we want to convert to RGB in uint8 and save it:
  imageeq =Image.fromarray((colored_image[:, :, :3] * 255).astype(np.uint8))
  return imageeq

def equalize_this(image_file, with_plot=False, gray_scale=False, bins=256):
    image_src = read_this(image_file=image_file, gray_scale=gray_scale)
    if not gray_scale:
        r_image = image_src[:, :, 0]
        g_image = image_src[:, :, 1]
        b_image = image_src[:, :, 2]

        r_image_eq = enhance_contrast(image_matrix=r_image)
        g_image_eq = enhance_contrast(image_matrix=g_image)
        b_image_eq = enhance_contrast(image_matrix=b_image)

        image_eq = np.dstack(tup=(r_image_eq, g_image_eq, b_image_eq))
        cmap_val = None
    else:
        image_eq = enhance_contrast(image_matrix=image_src)
        cmap_val = 'gray'

    if with_plot:
        fig = plt.figure(figsize=(10, 20))

        ax1 = fig.add_subplot(2, 2, 1)
        ax1.axis("off")
        ax1.title.set_text('Original')
        ax2 = fig.add_subplot(2, 2, 2)
        ax2.axis("off")
        ax2.title.set_text("Equalized")

        ax1.imshow(image_src, cmap=cmap_val)
        plt.imsave('test.png', image_src, cmap=cmap_val)
        image_eq=ax2.imshow(image_eq, cmap=cmap_val)
        plt.imsave('test.png', image_src, cmap=cmap_val)
        from PIL import Image
        from numpy import asarray
        # load the image
        image = Image.open('test.png')
        # convert image to numpy array
        data = asarray(image)
        print(type(data))
        # summarize shape
        print(data.shape)

        # create Pillow image
        image2 = Image.fromarray(data)
        print(type(image2))

        # summarize image details
        print(image2.mode)
        print(image2.size)

        return True
    print(type(image2))
    return image2
a=equalize_this(image_file='/content/drive/MyDrive/retinopathy-dataset/nosymptoms/10265_left.jpeg', with_plot=True)
print(a)
path="C:\\Somewhere\\myproperdirectory\\"
#from skimage.io._plugins.pil_plugin import ndarray_to_pil, pil_to_ndarray
#im=ndarray_to_pil(a).convert("1")
#a.save("your_file.jpeg")
#data = np.asarray( img, dtype="int32" )
#img = Image.fromarray( np.asarray( np.clip(data,0,255), dtype="uint8"), "L" )
#image1 = cv2.imread( 'test.png')
#input_arr = image.img_to_array(a)
input_arr = np.array([a])
print(type(input_arr))
input_arr=input_arr.flatten()
print(type(input_arr))

In [ ]:
from google.colab import files
#plt.savefig("/content/test.png")
files.upload()